In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sma

In [17]:
df = pd.read_csv("CleanedData.csv")
df = df[df["Price"] > 0]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1643 entries, 0 to 1642
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Company                 1643 non-null   object 
 1   Price                   1643 non-null   float64
 2   PriceStartDate          1643 non-null   object 
 3   Date Added              1643 non-null   object 
 4   InflationAdjustedPrice  1643 non-null   float64
 5   Analysis                1614 non-null   object 
 6   P or E                  1643 non-null   object 
 7   Pre2005Flag             1643 non-null   int64  
 8   PreviousPatents         1643 non-null   int64  
 9   LatestExpiration        1643 non-null   object 
 10  MonthsUntilExpiration   1643 non-null   float64
 11  PriceDelta              1643 non-null   float64
dtypes: float64(4), int64(2), object(6)
memory usage: 166.9+ KB


In [4]:
mean = df['InflationAdjustedPrice'].mean()
std =df['InflationAdjustedPrice'].std()
df['InflationAdjustedPriceZScore'] = (df['InflationAdjustedPrice'] - mean)/std
df= df[(df['InflationAdjustedPriceZScore'] <=3)&(df['InflationAdjustedPriceZScore'] >=-3)]

In [5]:
df['P or E'].value_counts(normalize=True)

P    0.806187
E    0.193813
Name: P or E, dtype: float64

Assigning binary dependent variable from the Analysis variable. The following categories will be classified as suspected evergreen:
- P:PED
- PTAorPTE
- DlistRequest
- NPP
- PED
- P-PEDExtension
- UCsamemonth
- DP
- DS/DP
- DS/DP/UCnew
- DP/UCnew
- DS

In [6]:
df['EvergreenFlag'] = [0] * len(df)
df.loc[df['P or E']=='E','EvergreenFlag'] = 1

In [7]:
df['EvergreenFlag'].value_counts(normalize=True)

0    0.806187
1    0.193813
Name: EvergreenFlag, dtype: float64

In [8]:
model_df = df[['InflationAdjustedPrice','MonthsUntilExpiration',
              'PreviousPatents','EvergreenFlag','PriceDelta']]
#model_df['Initial Filing'] = [0] * len(model_df)
#model_df.loc[model_df['PreviousPatents']==0,'Initial Filing'] = 1

In [9]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1584 entries, 0 to 1642
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   InflationAdjustedPrice  1584 non-null   float64
 1   MonthsUntilExpiration   1584 non-null   float64
 2   PreviousPatents         1584 non-null   int64  
 3   EvergreenFlag           1584 non-null   int64  
 4   PriceDelta              1584 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 74.2 KB


In [10]:
def ZStandardize(array):
    mean = array.mean()
    std = array.std()
    new_array = (array - mean)/std
    return(new_array)

In [11]:
for column in model_df.drop(columns='EvergreenFlag').columns:
    model_df[column] = ZStandardize(model_df[column])

C:\Users\sulli\AppData\Local\Temp\ipykernel_13104\770102775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[column] = ZStandardize(model_df[column])


In [12]:
model_df.describe()

,InflationAdjustedPrice,MonthsUntilExpiration,PreviousPatents,EvergreenFlag,PriceDelta
count,1.584000e+03,1.584000e+03,1.584000e+03,1584.000000,1.584000e+03
mean,3.059491e-15,4.243239e-16,-3.452485e-15,0.193813,5.237113e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,0.395409,1.000000e+00
min,-4.903532e-01,-2.899132e+00,-6.724874e-01,0.000000,-3.145871e+00
25%,-4.566217e-01,-7.671145e-01,-6.724874e-01,0.000000,-1.987420e-01
50%,-2.895706e-01,2.385543e-01,-4.052805e-01,0.000000,3.845834e-01
75%,-1.945880e-02,8.218422e-01,3.072710e-01,0.000000,4.445771e-01
max,5.851313e+00,1.807398e+00,4.627115e+00,1.000000,6.751824e+00


In [13]:
model_df['EvergreenFlag'].value_counts(normalize=True)

0    0.806187
1    0.193813
Name: EvergreenFlag, dtype: float64

In [14]:
model_df = sma.add_constant(model_df)
clf = sma.Logit(model_df['EvergreenFlag'],model_df.drop(columns='EvergreenFlag')).fit()
print(clf.summary())

Optimization terminated successfully.
         Current function value: 0.474972
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:          EvergreenFlag   No. Observations:                 1584
Model:                          Logit   Df Residuals:                     1579
Method:                           MLE   Df Model:                            4
Date:                Tue, 28 Mar 2023   Pseudo R-squ.:                 0.03403
Time:                        11:18:05   Log-Likelihood:                -752.36
converged:                       True   LL-Null:                       -778.86
Covariance Type:            nonrobust   LLR p-value:                 8.482e-11
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -1.4863      0.067    -22.247      0.000      -1.617      